# ChatGPT MODEL

In [ ]:
####

# PERPLEXITY MODEL

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.utils import save_image
from PIL import Image
import os

class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        self.encoder = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=4, stride=2, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
        )

        self.middle = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
        )

        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(64, 3, kernel_size=4, stride=2, padding=1),
            nn.Tanh()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        middle = self.middle(encoded)
        decoded = self.decoder(middle)
        return decoded

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.model = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(512, 1, kernel_size=4, stride=2, padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        output = self.model(x)
        return output

def preprocess_image(image_path, size):
    image = Image.open(image_path).convert("RGB")
    transform = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(20),
        transforms.RandomResizedCrop(size, scale=(0.8, 1.0)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
    return transform(image)

def load_dataset(bw_folder, color_folder, image_size, batch_size):
    bw_image_paths = [os.path.join(bw_folder, filename) for filename in os.listdir(bw_folder) if not filename.startswith('.')]
    color_image_paths = [os.path.join(color_folder, filename) for filename in os.listdir(color_folder) if not filename.startswith('.')]

    dataset = [(preprocess_image(bw_path, image_size), preprocess_image(color_path, image_size)) for bw_path, color_path in zip(bw_image_paths, color_image_paths)]
    train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    return train_loader

def train(generator, discriminator, train_loader, num_epochs, device):
    criterion_GAN = nn.MSELoss()
    criterion_cycle = nn.L1Loss()
    criterion_identity = nn.L1Loss()

    optimizer_G = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
    optimizer_D = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))

    for epoch in range(num_epochs):
        for batch_idx, (bw_imgs, color_imgs) in enumerate(train_loader):
            bw_imgs = bw_imgs.to(device)
            color_imgs = color_imgs.to(device)

            # Train the generator
            optimizer_G.zero_grad()

            fake_color_imgs = generator(bw_imgs)

            # Identity loss
            resized_color_imgs = F.interpolate(color_imgs, size=fake_color_imgs.size()[2:], mode='bilinear', align_corners=True)
            loss_identity = criterion_identity(fake_color_imgs, resized_color_imgs) * 5.0

            # GAN loss
            pred_fake = discriminator(fake_color_imgs)
            loss_GAN = criterion_GAN(pred_fake, torch.ones_like(pred_fake))

            # Cycle loss
            recovered_bw_imgs = generator(fake_color_imgs)
            loss_cycle = criterion_cycle(recovered_bw_imgs, bw_imgs) * 10.0

            # Total generator loss
            loss_G = loss_identity + loss_GAN + loss_cycle
            loss_G.backward()
            optimizer_G.step()

            # Train the discriminator
            optimizer_D.zero_grad()

            pred_real = discriminator(color_imgs)
            loss_real = criterion_GAN(pred_real, torch.ones_like(pred_real))

            pred_fake = discriminator(fake_color_imgs.detach())
            loss_fake = criterion_GAN(pred_fake, torch.zeros_like(pred_fake))

            # Total discriminator loss
            loss_D = (loss_real + loss_fake) * 0.5
            loss_D.backward()
            optimizer_D.step()

            print(f"Epoch {epoch+1}/{num_epochs}, Batch {batch_idx}/{len(train_loader)}, Loss G: {loss_G.item()}, Loss D: {loss_D.item()}")

            if batch_idx % 100 == 0:
                save_image(fake_color_imgs.data[:25], f"output/generated_{epoch}_{batch_idx}.png", nrow=5, normalize=True)
                torch.save(generator.state_dict(), f"output/generator_{epoch}_{batch_idx}.pth")
                torch.save(discriminator.state_dict(), f"output/discriminator_{epoch}_{batch_idx}.pth")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
image_size = (256, 256)
batch_size = 8
num_epochs = 200

bw_folder = "/Users/zachariahalzubi/Desktop/onePieceColoring/black_and_white"
color_folder = "/Users/zachariahalzubi/Desktop/onePieceColoring/colored"

train_loader = load_dataset(bw_folder, color_folder, image_size, batch_size)

generator = Generator().to(device)
discriminator = Discriminator().to(device)
os.makedirs("output", exist_ok=True)
train(generator, discriminator, train_loader, num_epochs, device)

Epoch 1/200, Batch 0/17, Loss G: 13.144043922424316, Loss D: 0.2546835243701935
Epoch 1/200, Batch 1/17, Loss G: 12.996423721313477, Loss D: 0.23017817735671997
Epoch 1/200, Batch 2/17, Loss G: 12.391427993774414, Loss D: 0.2005060762166977
Epoch 1/200, Batch 3/17, Loss G: 11.514909744262695, Loss D: 0.17453822493553162
Epoch 1/200, Batch 4/17, Loss G: 10.995420455932617, Loss D: 0.17776291072368622
Epoch 1/200, Batch 5/17, Loss G: 10.745889663696289, Loss D: 0.21635034680366516
Epoch 1/200, Batch 6/17, Loss G: 10.020939826965332, Loss D: 0.17553022503852844
Epoch 1/200, Batch 7/17, Loss G: 9.9434814453125, Loss D: 0.17866140604019165
Epoch 1/200, Batch 8/17, Loss G: 9.279485702514648, Loss D: 0.16081073880195618
Epoch 1/200, Batch 9/17, Loss G: 9.317586898803711, Loss D: 0.13852867484092712
Epoch 1/200, Batch 10/17, Loss G: 9.077943801879883, Loss D: 0.1669844090938568
Epoch 1/200, Batch 11/17, Loss G: 8.874163627624512, Loss D: 0.15416814386844635
Epoch 1/200, Batch 12/17, Loss G: 8.

Epoch 7/200, Batch 1/17, Loss G: 6.514565467834473, Loss D: 0.16086213290691376
Epoch 7/200, Batch 2/17, Loss G: 6.63516902923584, Loss D: 0.18262633681297302
Epoch 7/200, Batch 3/17, Loss G: 6.302554130554199, Loss D: 0.20115520060062408
Epoch 7/200, Batch 4/17, Loss G: 6.819849014282227, Loss D: 0.1710727959871292
Epoch 7/200, Batch 5/17, Loss G: 6.556005477905273, Loss D: 0.11634686589241028
Epoch 7/200, Batch 6/17, Loss G: 6.711249351501465, Loss D: 0.08369147777557373
Epoch 7/200, Batch 7/17, Loss G: 6.562498569488525, Loss D: 0.11935809999704361
Epoch 7/200, Batch 8/17, Loss G: 6.74778938293457, Loss D: 0.1516401767730713
Epoch 7/200, Batch 9/17, Loss G: 6.316725254058838, Loss D: 0.10963739454746246
Epoch 7/200, Batch 10/17, Loss G: 6.536334037780762, Loss D: 0.13659654557704926
Epoch 7/200, Batch 11/17, Loss G: 6.634660720825195, Loss D: 0.269967645406723
Epoch 7/200, Batch 12/17, Loss G: 6.488336563110352, Loss D: 0.132921040058136
Epoch 7/200, Batch 13/17, Loss G: 6.959465503

Epoch 13/200, Batch 1/17, Loss G: 6.120887756347656, Loss D: 0.088837169110775
Epoch 13/200, Batch 2/17, Loss G: 5.816176414489746, Loss D: 0.08620739728212357
Epoch 13/200, Batch 3/17, Loss G: 6.410170555114746, Loss D: 0.06559615582227707
Epoch 13/200, Batch 4/17, Loss G: 5.722382545471191, Loss D: 0.08213654160499573
Epoch 13/200, Batch 5/17, Loss G: 6.10139799118042, Loss D: 0.10776391625404358
Epoch 13/200, Batch 6/17, Loss G: 5.891660690307617, Loss D: 0.08158250898122787
Epoch 13/200, Batch 7/17, Loss G: 6.164983749389648, Loss D: 0.14620986580848694
Epoch 13/200, Batch 8/17, Loss G: 5.693112373352051, Loss D: 0.11549228429794312
Epoch 13/200, Batch 9/17, Loss G: 5.906113624572754, Loss D: 0.11566787213087082
Epoch 13/200, Batch 10/17, Loss G: 5.747712135314941, Loss D: 0.09769114851951599
Epoch 13/200, Batch 11/17, Loss G: 5.990267276763916, Loss D: 0.08414775133132935
Epoch 13/200, Batch 12/17, Loss G: 5.832071304321289, Loss D: 0.07994012534618378
Epoch 13/200, Batch 13/17, L

Epoch 19/200, Batch 0/17, Loss G: 5.579339981079102, Loss D: 0.052514709532260895
Epoch 19/200, Batch 1/17, Loss G: 5.7586894035339355, Loss D: 0.07752960920333862
Epoch 19/200, Batch 2/17, Loss G: 5.605216026306152, Loss D: 0.07374760508537292
Epoch 19/200, Batch 3/17, Loss G: 5.511015892028809, Loss D: 0.06525661796331406
Epoch 19/200, Batch 4/17, Loss G: 5.744221210479736, Loss D: 0.09079723060131073
Epoch 19/200, Batch 5/17, Loss G: 5.1043901443481445, Loss D: 0.13373063504695892
Epoch 19/200, Batch 6/17, Loss G: 5.786962509155273, Loss D: 0.09288973361253738
Epoch 19/200, Batch 7/17, Loss G: 5.644351482391357, Loss D: 0.028270989656448364
Epoch 19/200, Batch 8/17, Loss G: 5.459386825561523, Loss D: 0.05657610297203064
Epoch 19/200, Batch 9/17, Loss G: 5.625794887542725, Loss D: 0.04330229014158249
Epoch 19/200, Batch 10/17, Loss G: 5.46687650680542, Loss D: 0.06232664734125137
Epoch 19/200, Batch 11/17, Loss G: 5.266922950744629, Loss D: 0.10014835000038147
Epoch 19/200, Batch 12/

Epoch 24/200, Batch 16/17, Loss G: 5.611673831939697, Loss D: 0.025806788355112076
Epoch 25/200, Batch 0/17, Loss G: 5.155540466308594, Loss D: 0.050159480422735214
Epoch 25/200, Batch 1/17, Loss G: 5.319460391998291, Loss D: 0.02642800286412239
Epoch 25/200, Batch 2/17, Loss G: 5.462569236755371, Loss D: 0.03076520934700966
Epoch 25/200, Batch 3/17, Loss G: 5.245382308959961, Loss D: 0.02987590618431568
Epoch 25/200, Batch 4/17, Loss G: 5.407851696014404, Loss D: 0.029772482812404633
Epoch 25/200, Batch 5/17, Loss G: 5.168210029602051, Loss D: 0.04519045352935791
Epoch 25/200, Batch 6/17, Loss G: 5.392905235290527, Loss D: 0.07675552368164062
Epoch 25/200, Batch 7/17, Loss G: 5.1284894943237305, Loss D: 0.1075509637594223
Epoch 25/200, Batch 8/17, Loss G: 5.5686774253845215, Loss D: 0.14393827319145203
Epoch 25/200, Batch 9/17, Loss G: 5.319249629974365, Loss D: 0.03771303594112396
Epoch 25/200, Batch 10/17, Loss G: 5.315615653991699, Loss D: 0.030994873493909836
Epoch 25/200, Batch 1

...

Epoch 177/200, Batch 4/17, Loss G: 3.6723697185516357, Loss D: 0.002637745812535286
Epoch 177/200, Batch 5/17, Loss G: 3.779456377029419, Loss D: 0.0013681005220860243
Epoch 177/200, Batch 6/17, Loss G: 3.7773597240448, Loss D: 0.005186197813600302
Epoch 177/200, Batch 7/17, Loss G: 3.7173166275024414, Loss D: 0.011059457436203957
Epoch 177/200, Batch 8/17, Loss G: 3.636151075363159, Loss D: 0.00261456030420959
Epoch 177/200, Batch 9/17, Loss G: 3.719331741333008, Loss D: 0.004737219773232937
Epoch 177/200, Batch 10/17, Loss G: 3.781048536300659, Loss D: 0.006111695896834135
Epoch 177/200, Batch 11/17, Loss G: 3.938969135284424, Loss D: 0.003988390788435936
Epoch 177/200, Batch 12/17, Loss G: 3.7538208961486816, Loss D: 0.0015458648558706045
Epoch 177/200, Batch 13/17, Loss G: 3.8964691162109375, Loss D: 0.003130412194877863
Epoch 177/200, Batch 14/17, Loss G: 3.6919682025909424, Loss D: 0.0028313444927334785
Epoch 177/200, Batch 15/17, Loss G: 3.869879722595215, Loss D: 0.001372031751

Epoch 183/200, Batch 0/17, Loss G: 3.7182955741882324, Loss D: 0.0035513064358383417
Epoch 183/200, Batch 1/17, Loss G: 3.7048254013061523, Loss D: 0.007198923267424107
Epoch 183/200, Batch 2/17, Loss G: 3.8078837394714355, Loss D: 0.031031476333737373
Epoch 183/200, Batch 3/17, Loss G: 3.4689717292785645, Loss D: 0.054298464208841324
Epoch 183/200, Batch 4/17, Loss G: 3.887603282928467, Loss D: 0.006422160193324089
Epoch 183/200, Batch 5/17, Loss G: 3.7834086418151855, Loss D: 0.011645939201116562
Epoch 183/200, Batch 6/17, Loss G: 3.791487693786621, Loss D: 0.0012764029670506716
Epoch 183/200, Batch 7/17, Loss G: 3.6849215030670166, Loss D: 0.004694134928286076
Epoch 183/200, Batch 8/17, Loss G: 3.5897130966186523, Loss D: 0.0021422808058559895
Epoch 183/200, Batch 9/17, Loss G: 3.5752758979797363, Loss D: 0.006246720906347036
Epoch 183/200, Batch 10/17, Loss G: 3.7369370460510254, Loss D: 0.004128226079046726
Epoch 183/200, Batch 11/17, Loss G: 3.717252731323242, Loss D: 0.003716083

Epoch 188/200, Batch 13/17, Loss G: 3.8534607887268066, Loss D: 0.020218877121806145
Epoch 188/200, Batch 14/17, Loss G: 3.5410847663879395, Loss D: 0.022355714812874794
Epoch 188/200, Batch 15/17, Loss G: 3.722806930541992, Loss D: 0.00954044796526432
Epoch 188/200, Batch 16/17, Loss G: 3.9457836151123047, Loss D: 0.015416326001286507
Epoch 189/200, Batch 0/17, Loss G: 3.765031337738037, Loss D: 0.0037101060152053833
Epoch 189/200, Batch 1/17, Loss G: 3.594924211502075, Loss D: 0.019442973658442497
Epoch 189/200, Batch 2/17, Loss G: 3.691051959991455, Loss D: 0.01127806305885315
Epoch 189/200, Batch 3/17, Loss G: 3.7132742404937744, Loss D: 0.010845760814845562
Epoch 189/200, Batch 4/17, Loss G: 3.6200788021087646, Loss D: 0.016957396641373634
Epoch 189/200, Batch 5/17, Loss G: 3.815310001373291, Loss D: 0.005102886352688074
Epoch 189/200, Batch 6/17, Loss G: 3.6554148197174072, Loss D: 0.013583659194409847
Epoch 189/200, Batch 7/17, Loss G: 3.4853129386901855, Loss D: 0.0040664719417

Epoch 194/200, Batch 9/17, Loss G: 3.6206982135772705, Loss D: 0.008514698594808578
Epoch 194/200, Batch 10/17, Loss G: 3.5458412170410156, Loss D: 0.005737924482673407
Epoch 194/200, Batch 11/17, Loss G: 3.7980499267578125, Loss D: 0.001375660765916109
Epoch 194/200, Batch 12/17, Loss G: 3.563847541809082, Loss D: 0.0032285614870488644
Epoch 194/200, Batch 13/17, Loss G: 3.6870410442352295, Loss D: 0.0020482041873037815
Epoch 194/200, Batch 14/17, Loss G: 3.696399688720703, Loss D: 0.002757232403382659
Epoch 194/200, Batch 15/17, Loss G: 3.655787229537964, Loss D: 0.0034587751142680645
Epoch 194/200, Batch 16/17, Loss G: 3.658210039138794, Loss D: 0.0029142575804144144
Epoch 195/200, Batch 0/17, Loss G: 3.8092217445373535, Loss D: 0.0023223161697387695
Epoch 195/200, Batch 1/17, Loss G: 3.723594903945923, Loss D: 0.0014968004543334246
Epoch 195/200, Batch 2/17, Loss G: 3.639802932739258, Loss D: 0.004043163266032934
Epoch 195/200, Batch 3/17, Loss G: 3.5511512756347656, Loss D: 0.0009

Epoch 200/200, Batch 5/17, Loss G: 3.657466173171997, Loss D: 0.00466145109385252
Epoch 200/200, Batch 6/17, Loss G: 3.6123080253601074, Loss D: 0.0025287370663136244
Epoch 200/200, Batch 7/17, Loss G: 3.5896005630493164, Loss D: 0.0017605919856578112
Epoch 200/200, Batch 8/17, Loss G: 3.5299763679504395, Loss D: 0.0009864085586741567
Epoch 200/200, Batch 9/17, Loss G: 3.6142935752868652, Loss D: 0.0040749660693109035
Epoch 200/200, Batch 10/17, Loss G: 3.72916316986084, Loss D: 0.0013421502662822604
Epoch 200/200, Batch 11/17, Loss G: 3.6513428688049316, Loss D: 0.004924968350678682
Epoch 200/200, Batch 12/17, Loss G: 3.5499281883239746, Loss D: 0.0016350150108337402
Epoch 200/200, Batch 13/17, Loss G: 3.560455799102783, Loss D: 0.0010701711289584637
Epoch 200/200, Batch 14/17, Loss G: 3.535938262939453, Loss D: 0.0025572404265403748
Epoch 200/200, Batch 15/17, Loss G: 3.534745216369629, Loss D: 0.0047152601182460785
Epoch 200/200, Batch 16/17, Loss G: 3.577332019805908, Loss D: 0.010

In [5]:
import os

# Get the path to the user's desktop
desktop_path = os.path.join(os.path.expanduser("~"), "Desktop")

# Save the weights to the desktop
weights_path = os.path.join(desktop_path, "saved_weights2.pth")
torch.save(generator.state_dict(), weights_path)

In [10]:
import torch
from torchvision import transforms
from PIL import Image
import numpy as np
import os

# Load the trained model
generator = Generator()
weights_path = os.path.join(os.path.expanduser("~"), "Desktop", "saved_weights2.pth")
generator.load_state_dict(torch.load(weights_path))
generator.eval()

# Preprocess the test image
test_image_path = os.path.join(os.path.expanduser("~"), "Desktop","onePieceColoring", "black_and_white", "image_14.jpg")
image_size = (256, 256)
transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
test_image = Image.open(test_image_path)
input_image = transform(test_image).unsqueeze(0)

# Pass the image through the model
with torch.no_grad():
    colorized_image = generator(input_image)

# Postprocess the colorized image
colorized_image = colorized_image.squeeze(0).permute(1, 2, 0)
colorized_image = (colorized_image + 1) / 2  # Denormalize the image

# Convert the tensor to a numpy array
colorized_image_np = colorized_image.cpu().numpy()

# Convert the numpy array to a PIL Image
colorized_image_pil = Image.fromarray((255 * colorized_image_np).astype(np.uint8))

# Save the colorized image
save_path = os.path.join(os.path.expanduser("~"), "Desktop", "colored_by_model3.jpg")
colorized_image_pil.save(save_path)